# Waveform display

The code below receives the data from a COM-port and displays it. Here we can see raw microphone data as well as the convolution result.

First we need to import required libraries and open COM port.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import serial

%matplotlib qt

ser = serial.Serial("COM3", 921600)

# set up plots
fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot([], [])
line2, = ax.plot([], [])
line3, = ax.plot([], [])
line4, = ax.plot([], [])

## Show convolution output

In [ ]:
# length of the "bit" of the sequence
# set this value multiplied by 4 in "p_gen_set_pulse_len" function in SDK project
seq_len = 2
line1.set_xdata(np.linspace(0, 255+(seq_len*13), 255+(seq_len*13))) 
line2.set_xdata(np.linspace(0, 255+(seq_len*13), 255+(seq_len*13)))
line3.set_xdata(np.linspace(0, 255+(seq_len*13), 255+(seq_len*13)))
line4.set_xdata(np.linspace(0, 255+(seq_len*13), 255+(seq_len*13)))

# define convolution sequence
pattern = np.array([[1] * seq_len if(int(i)==1) else [-1] * seq_len for i in bin(0x1f35)[2:]]).flatten()

while True:
    echo = []
    # data receiving
    for i in range(4):
        idata = [int.from_bytes(ser.read(3), byteorder='big', signed="True") for x in range(256)]
        qdata = [int.from_bytes(ser.read(3), byteorder='big', signed="True") for x in range(256)]

        # convolution
        micdata = list(map(lambda x: x[0]+(x[1]*1j), list(zip(idata, qdata))))
        conv = np.convolve(micdata, pattern)

        # convert to amplitude and phase
        amp = [np.sqrt(x.imag**2 + x.real**2) for x in conv]
        phase = [np.arctan(x.real/x.imag) for x in conv]

        echo.append([amp, phase])

    # display result
    line1.set_ydata(echo[0][0])
    line2.set_ydata(echo[1][0])
    line3.set_ydata(echo[2][0])
    line4.set_ydata(echo[3][0])
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()

## Show raw output

In [ ]:
line1.set_xdata(np.linspace(0, 256, 256))
line2.set_xdata(np.linspace(0, 256, 256))
line3.set_xdata(np.linspace(0, 256, 256))
line4.set_xdata(np.linspace(0, 256, 256))

while True:
    echo = []
    # data receiving
    for i in range(4):
        idata = [int.from_bytes(ser.read(3), byteorder='big', signed="True") for x in range(256)]
        qdata = [int.from_bytes(ser.read(3), byteorder='big', signed="True") for x in range(256)]

        echo.append([idata, qdata])

    # display result
    line1.set_ydata(echo[0][0])
    line2.set_ydata(echo[1][0])
    line3.set_ydata(echo[2][0])
    line4.set_ydata(echo[3][0])
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()